# Tarea Final - Machine Learning

Alumno: Guillermo Lodeiro

Profesores: Jose Angel Carballo, Luis Garmendia, María José Gómez

Institución: Universidad Complutense de Madrid

Al correr Pycaret en mi data preprocesada, la mejor opción encontrada fue Random Forest. De esta forma, fui armando mi proceso de prueba de modelo. Comienzo por utilizar Randomized Search de sklearn, herramienta que fue muy útil para encontrar los hiperparámeteros optimos. En esta parte tambien me interiorice con los parámetros de esta herramienta, como n_jobs para utilizar el máximo rendimiento de mi pc. En cuanto a medir el rendimiento de mi modelo, opte por utilizar cross validation de sklearn y feature importances para analizar el peso de cada variable en el modelo.

In [1]:
import pandas as pd

In [2]:
data_train = pd.read_csv("training_set_values_pp6.csv")

In [3]:
data_test = pd.read_csv("test_set_values_pp6.csv")

In [4]:
data_labels = data_train["status_group"]
data_train.drop(['status_group'], axis=1, inplace=True)

In [5]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Define el espacio de búsqueda para los hiperparámetros
param_dist = {
    'n_estimators': randint(500, 1500),
    'max_depth': randint(15, 25),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2']
}

# Inicializa la búsqueda aleatoria con validación cruzada
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=100,  # Número de iteraciones (muestreo de hiperparámetros)
    cv=5,        # Número de pliegues de validación cruzada
    random_state=42,
    n_jobs=-1    # Utiliza todos los núcleos de CPU disponibles
)

# Entrena el modelo con el conjunto de datos
random_search.fit(data_train, data_labels)

# Muestra los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", random_search.best_params_)
 

Mejores hiperparámetros: {'max_depth': 23, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 1396}


In [21]:
from sklearn.ensemble import RandomForestClassifier

# Inicializar clasificador
rf_classifier = RandomForestClassifier(max_depth= 23, max_features= 'log2', min_samples_leaf= 1, min_samples_split= 3,
                                       n_estimators= 1396, random_state=42)

# Entrenar clasificador
rf_classifier.fit(data_train, data_labels)

# Realizar predicciones en el conjunto de prueba
predictions = rf_classifier.predict(data_test)

In [22]:
from sklearn.model_selection import cross_val_score

# Realiza la validación cruzada con 5 particiones
scores = cross_val_score(rf_classifier, data_train, data_labels, cv=5)

# Muestra los puntajes de validación cruzada
print("Puntajes de validación cruzada:", scores)
print("Precisión media:", scores.mean())

Puntajes de validación cruzada: [0.79234366 0.78695411 0.80827455 0.84606848 0.85042803]
Precisión media: 0.8168137670842889


In [23]:
# Obtener la importancia de las características
importances = rf_classifier.feature_importances_

# Crear un DataFrame para mostrar las importancias junto con el nombre de las características
feature_importances = pd.DataFrame(importances, index=data_train.columns, columns=['Importance'])

# Ordenar las características por su importancia de mayor a menor
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Mostrar las características ordenadas por su importancia
feature_importances.head(20)

,Importance
distance,0.076934
longitude,0.075792
latitude,0.074914
quantity_group_dry,0.074841
date_recorded,0.050378
ward,0.037161
population,0.034485
waterpoint_type_group_other,0.031442
quantity_group_enough,0.031373
extraction_type_class_other,0.027186


In [24]:
predictions

array([2, 0, 0, ..., 0, 0, 2], dtype=int64)

In [25]:
predictions_rf = pd.DataFrame(predictions, columns=['status_group'])

In [26]:
predictions_rf.value_counts()

status_group
0               9187
2               5048
1                615
Name: count, dtype: int64

In [27]:
columnas_train = pd.read_csv('Test_Set_Values.csv')

In [28]:
columna_id = columnas_train['id']

In [29]:
df_columna_id = pd.DataFrame(columna_id)

In [30]:
intento_rf = pd.concat([df_columna_id['id'], predictions_rf['status_group']], axis=1)

In [31]:
intento_rf

,id,status_group
0,50785,2
1,51630,0
2,17168,0
3,45559,2
4,49871,0
...,...,...
14845,39307,2
14846,18990,0
14847,28749,0
14848,33492,0


In [32]:
intento_rf['status_group'] = intento_rf['status_group'].replace({2: 'non functional', 1: 'functional needs repair', 0: 'functional'})

In [33]:
intento_rf.to_csv("random_forest_pp6_smote.csv", index = False)